In [7]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI,ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import os

In [14]:
embeddings = OpenAIEmbeddings()

In [18]:
llm = ChatOpenAI(base_url=os.getenv('OPENAI_BASE_URL'),temperature=0)

In [11]:
loader = TextLoader("../tests/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)


docsearch = Chroma.from_documents(texts, embeddings)



In [19]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(),verbose=True)

In [17]:
query = "What did the president say about Ketanji Brown Jackson"
qa.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What did the president say about Ketanji Brown Jackson',
 'result': "The President mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, describing her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [20]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [21]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(),verbose=True,return_source_documents=True,chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [28]:
query = "What did the president say about Ketanji Brown Jackson"
qa.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What did the president say about Ketanji Brown Jackson',
 'result': 'The President described Ketanji Brown Jackson as "one of our nation’s top legal minds" who will continue Justice Breyer’s legacy of excellence.',
 'source_documents': [Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal

In [27]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="map_reduce", 
                                 retriever=docsearch.as_retriever(),
                                 verbose=True,
                                 return_source_documents=True,
                              
                                 )



In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
qa.invoke(query)